# OpenVINO

Dalam buku catatan ini, kami akan menunjukkan cara menggunakan perangkat OpenVINO untuk menerapkan model pembelajaran mendalam pada perangkat edge dan mengkuantisasi model untuk mengurangi ukuran model dan latensi inferensi. Kami akan melatih model CNN sederhana pada dataset MNIST, mengonversinya ke format OpenVINO IR, dan mengkuantisasi model ke presisi INT8. Kami kemudian akan membandingkan ukuran dan kinerja model terkuantisasi dengan model FP32 asli.

## Siapkan OpenVINO

Pertama, kita perlu menginstal OpenVINO, NNCF, dan torch

In [1]:
%pip install -q "openvino>=2023.1.0" torch torchvision --extra-index-url https://download.pytorch.org/whl/cpu
%pip install -q "nncf>=2.6.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.4 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pathlib
import numpy as np
import openvino as ov
import nncf

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, openvino


## Melatih Model

Selanjutnya, tentukan dan latih model CNN sederhana pada dataset MNIST

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

train_dataset = datasets.MNIST('./data', train=True, download=True,transform=transform)
test_dataset = datasets.MNIST('./data', train=False,transform=transform)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=12, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(12 * 13 * 13, 10)

    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        output = F.log_softmax(x, dim=1)
        return output


train_loader = torch.utils.data.DataLoader(train_dataset, 32)
test_loader = torch.utils.data.DataLoader(test_dataset, 32)

device = "cpu"

epochs = 1

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()

for epoch in range(1, epochs+1):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))

MODEL_DIR = pathlib.Path("./models")
MODEL_DIR.mkdir(exist_ok=True)
torch.save(model.state_dict(), MODEL_DIR / "original_model.p")

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 37.8MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.37MB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 10.8MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 7.93MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.361235
Train Epoch: 1 [32/60000 (0%)]	Loss: 2.350525
Train Epoch: 1 [64/60000 (0%)]	Loss: 2.055931
Train Epoch: 1 [96/60000 (0%)]	Loss: 1.860162
Train Epoch: 1 [128/60000 (0%)]	Loss: 2.009008
Train Epoch: 1 [160/60000 (0%)]	Loss: 2.025581
Train Epoch: 1 [192/60000 (0%)]	Loss: 1.507131
Train Epoch: 1 [224/60000 (0%)]	Loss: 1.683474
Train Epoch: 1 [256/60000 (0%)]	Loss: 1.674473
Train Epoch: 1 [288/60000 (0%)]	Loss: 1.344969
Train Epoch: 1 [320/60000 (1%)]	Loss: 1.399575
Train Epoch: 1 [352/60000 (1%)]	Loss: 1.257095
Train Epoch: 1 [384/60000 (1%)]	Loss: 1.258982
Train Epoch: 1 [416/60000 (1%)]	Loss: 1.130673
Train Epoch: 1 [448/60000 (1%)]	Loss: 1.063616
Train Epoch: 1 [480/60000 (1%)]	Loss: 1.435441
Train Epoch: 1 [512/60000 (1%)]	Loss: 1.314874
Train Epoch: 1 [544/60000 (1%)]	Loss: 0.928313
Train Epoch: 1 [576/60000 (1%)]	Loss: 1.237929
Train Epoch: 1 [608/60000 (1%)]	Loss:

## Konversi ke OpenVINO IR

Kemudian, konversi model ke format OpenVINO IR

In [4]:
core = ov.Core()
example_input = next(iter(test_loader))[0]
ov_model = ov.convert_model(model, example_input=example_input)
ov.save_model(ov_model, MODEL_DIR / f"openvino_ir.xml")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


## Kuantisasi

Untuk mengkuantisasi model menggunakan NNCF, pertama-tama, buat fungsi transformasi untuk mengonversi tensor torch ke array NumPy, lalu gunakan fungsi yang dibuat bersama dengan pemuat data pytorch untuk membuat set data kalibrasi menggunakan kelas `Dataset` dari NNCF. Selanjutnya, kuantisasi model menggunakan fungsi `quantize` dari NNCF. Terakhir, kompilasi model yang dikuantisasi dan simpan sebagai format OpenVINO IR.

In [5]:
def transform_fn(data_item):
    images, _ = data_item
    return images.numpy()

calibration_dataset = nncf.Dataset(train_loader, transform_fn)
quantized_model = nncf.quantize(ov_model, calibration_dataset)
model_int8 = ov.compile_model(quantized_model)
input_fp32 = next(iter(test_loader))[0][0:1]
res = model_int8(input_fp32)
ov.save_model(quantized_model, MODEL_DIR / f"quant_openvino_ir.xml")

Output()

Output()

## Periksa Ukuran

Bandingkan ukuran model FP32 dan INT8

In [6]:
%ls -lh {MODEL_DIR}

total 176K
-rw-r--r-- 1 root root 40K Jan  4 06:17 openvino_ir.bin
-rw-r--r-- 1 root root 11K Jan  4 06:17 openvino_ir.xml
-rw-r--r-- 1 root root 82K Jan  4 06:17 original_model.p
-rw-r--r-- 1 root root 21K Jan  4 06:17 quant_openvino_ir.bin
-rw-r--r-- 1 root root 16K Jan  4 06:17 quant_openvino_ir.xml


## Periksa Akurasi

Evaluasi akurasi model INT8 dan bandingkan dengan model FP32

In [7]:
def test_ov(model, data_loader):
    compiled_model = ov.compile_model(model)
    test_loss = 0
    correct = 0
    for data, target in data_loader:
        output = torch.tensor(compiled_model(data)[0])
        test_loss += F.nll_loss(output, target, reduction='sum').item()  # jumlahkan loss batch
        pred = output.argmax(dim=1, keepdim=True)  # dapatkan indeks probabilitas log maksimum
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(data_loader.dataset)

    return 100. * correct / len(data_loader.dataset)

# Menghitung akurasi model asli
acc = test_ov(ov_model, test_loader)
print(f"Akurasi model asli: {acc}")

# Menghitung akurasi model yang telah dikuantisasi
qacc = test_ov(quantized_model, test_loader)
print(f"Akurasi model yang telah dikuantisasi: {qacc}")


Akurasi model asli: 96.37
Akurasi model yang telah dikuantisasi: 96.76
